In [1]:
import os
import pandas as pd

# Define the folder containing CSV files
folder_path = "Eredivisie 2024-2025/xgCSV"  # Change this to your folder path
output_file = "merged_output.xlsx"  # Name of the output Excel file

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through CSV files and read them
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)  # Read CSV
    df["source_file"] = file  # Add column to track source file
    dataframes.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(dataframes, ignore_index=True)

# Save to an Excel file
merged_df.to_excel(output_file, index=False)

print(f"Merged {len(csv_files)} CSV files into {output_file}")


Merged 203 CSV files into merged_output.xlsx


In [18]:
import pandas as pd

# Define the Excel file to load
excel_file = "merged_output.xlsx"

# Load the Excel file
df = pd.read_excel(excel_file)

# Strip column names of extra spaces
df.columns = df.columns.str.strip()

# Expected columns
required_columns = {"PlayerId", "relatedPlayerId", "xG", "PsxG", "isGoal"}

# Find missing columns
missing_columns = required_columns - set(df.columns)
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    print("Available columns in dataset:", df.columns.tolist())
else:
    # Step 1: Compute total xG per PlayerId (ensuring uniqueness)
    xg_per_player = df.groupby("PlayerId", as_index=False)["xG"].sum()

    # Step 2: Compute xAG - sum of xG where PlayerId appears as relatedPlayerId
    xag_per_related = df.groupby("relatedPlayerId", as_index=False)["xG"].sum().rename(columns={"relatedPlayerId": "PlayerId", "xG": "xAG"})

    # Ensure the xAG column is added properly
    df = df.merge(xag_per_related, on="PlayerId", how="left").fillna(0)
    
    # Debug: Check if xAG column is correctly added
    print("First few rows of the dataset with xAG:\n", df.head())

    # Step 3: Count the number of shots per PlayerId (i.e., non-null xG values)
    shots_per_player = df.groupby("PlayerId", as_index=False)["xG"].count().rename(columns={"xG": "Shots"})

    # Step 4: Count goals (where isGoal is True)
    goals_per_player = df[df["isGoal"] == True].groupby("PlayerId", as_index=False).size().rename(columns={"size": "Goals"})

    # Step 5: Count shot assists (where relatedPlayerId has a valid xAG)
    shot_assists_per_player = df[df["xAG"] > 0].groupby("relatedPlayerId", as_index=False).size().rename(columns={"size": "Shot Assists"})

    # Step 6: Count assists (where both xAG and isGoal are True)
    assists_per_player = df[(df["xAG"] > 0) & (df["isGoal"] == True)].groupby("relatedPlayerId", as_index=False).size().rename(columns={"size": "Assists"})

    # Step 7: Merge all the necessary information
    summary_df = df.groupby("PlayerId", as_index=False)[["xG", "PsxG"]].sum()
    summary_df = summary_df.merge(xag_per_related, on="PlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(shots_per_player, on="PlayerId", how="left")
    summary_df = summary_df.merge(goals_per_player, on="PlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(shot_assists_per_player, left_on="PlayerId", right_on="relatedPlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(assists_per_player, left_on="PlayerId", right_on="relatedPlayerId", how="left").fillna(0)

    # Drop the unnecessary relatedPlayerId column only if it exists
    if "relatedPlayerId" in summary_df.columns:
        summary_df.drop(columns=["relatedPlayerId"], inplace=True)

    # Debug: Check the merged summary
    print("Final summary data:\n", summary_df)

    # Save summary to Excel
    with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a") as writer:
        summary_df.to_excel(writer, sheet_name="Player Summary 5", index=False)

    print(f"✅ Summary saved to {excel_file} under 'Player Summary 3' sheet.")


First few rows of the dataset with xAG:
    Unnamed: 0        xG                      Date    PlayerId         TeamId  \
0           1  0.101616  2025-02-01T19:10:46.511Z    L. Smans  SC Heerenveen   
1           2  0.080696  2025-02-01T19:13:20.541Z   E. van Ee  SC Heerenveen   
2           3  0.089958  2025-02-01T19:13:23.317Z   E. van Ee  SC Heerenveen   
3           4  0.043790  2025-02-01T19:13:26.075Z  I. Sebaoui  SC Heerenveen   
4           5  0.044089  2025-02-01T19:14:53.877Z    L. Smans  SC Heerenveen   

        HomeTeam         AwayTeam  timeMin  timeSec     x  ...  isOwnGoal  \
0  SC Heerenveen  Fortuna Sittard        7       59  88.3  ...      False   
1  SC Heerenveen  Fortuna Sittard       10       33  83.1  ...      False   
2  SC Heerenveen  Fortuna Sittard       10       36  86.5  ...      False   
3  SC Heerenveen  Fortuna Sittard       10       38  79.3  ...      False   
4  SC Heerenveen  Fortuna Sittard       12        6  87.3  ...      False   

   expandedMinu

In [20]:
import pandas as pd

# Define the new Excel file to save the results
new_excel_file = "player_summary_output.xlsx"

# Load the original Excel file
excel_file = "merged_output.xlsx"
df = pd.read_excel(excel_file)

# Strip column names of extra spaces
df.columns = df.columns.str.strip()

# Expected columns
required_columns = {"PlayerId", "relatedPlayerId", "xG", "PsxG", "isGoal"}

# Find missing columns
missing_columns = required_columns - set(df.columns)
if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    print("Available columns in dataset:", df.columns.tolist())
else:
    # Step 1: Compute total xG per PlayerId (ensuring uniqueness)
    xg_per_player = df.groupby("PlayerId", as_index=False)["xG"].sum()

    # Step 2: Compute xAG - sum of xG where PlayerId appears as relatedPlayerId
    xag_per_related = df.groupby("relatedPlayerId", as_index=False)["xG"].sum().rename(columns={"relatedPlayerId": "PlayerId", "xG": "xAG"})

    # Ensure the xAG column is added properly
    df = df.merge(xag_per_related, on="PlayerId", how="left").fillna(0)
    
    # Step 3: Count the number of shots per PlayerId (i.e., non-null xG values)
    shots_per_player = df.groupby("PlayerId", as_index=False)["xG"].count().rename(columns={"xG": "Shots"})

    # Step 4: Count goals (where isGoal is True)
    goals_per_player = df[df["isGoal"] == True].groupby("PlayerId", as_index=False).size().rename(columns={"size": "Goals"})

    # Step 5: Count shot assists (where relatedPlayerId has a valid xAG)
    shot_assists_per_player = df[df["xAG"] > 0].groupby("relatedPlayerId", as_index=False).size().rename(columns={"size": "Shot Assists"})

    # Step 6: Count assists (where both xAG and isGoal are True)
    assists_per_player = df[(df["xAG"] > 0) & (df["isGoal"] == True)].groupby("relatedPlayerId", as_index=False).size().rename(columns={"size": "Assists"})

    # Step 7: Merge all the necessary information
    summary_df = df.groupby("PlayerId", as_index=False)[["xG", "PsxG"]].sum()
    summary_df = summary_df.merge(xag_per_related, on="PlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(shots_per_player, on="PlayerId", how="left")
    summary_df = summary_df.merge(goals_per_player, on="PlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(shot_assists_per_player, left_on="PlayerId", right_on="relatedPlayerId", how="left").fillna(0)
    summary_df = summary_df.merge(assists_per_player, left_on="PlayerId", right_on="relatedPlayerId", how="left").fillna(0)

    # Step 8: Drop the _x and _y suffixed columns for relatedPlayerId
    columns_to_drop = [col for col in summary_df.columns if 'relatedPlayerId' in col and col != 'relatedPlayerId']
    summary_df.drop(columns=columns_to_drop, inplace=True)

    # Debug: Check the merged summary
    print("Final summary data:\n", summary_df)

    # Save summary to a new Excel file without specifying the sheet name
    with pd.ExcelWriter(new_excel_file, engine="openpyxl") as writer:
        summary_df.to_excel(writer, index=False)  # No sheet name specified

    print(f"✅ Summary saved to {new_excel_file} without sheet name.")


Final summary data:
            PlayerId        xG      PsxG       xAG  Shots  Goals  Shot Assists  \
0      A. Adelgaard  0.462190  0.533075  0.769098      4    0.0          11.0   
1          A. Blake  0.260492  0.296990  1.194493      4    2.0           6.0   
2          A. Condé  0.100701  0.117747  0.414965      2    0.0           6.0   
3       A. Descotte  1.958818  2.346026  0.085703      9    2.0           1.0   
4     A. El Azzouzi  2.127889  2.492030  0.848910     22    0.0          14.0   
..              ...       ...       ...       ...    ...    ...           ...   
387    Zidane Iqbal  0.545756  0.576947  0.512838     11    1.0           9.0   
388  Àlex Carbonell  0.085703  0.102844  0.789865      1    0.0           8.0   
389     Álex Balboa  0.436147  0.482757  0.154940      8    1.0           2.0   
390       É. Michut  0.617109  0.711251  0.032533      4    1.0           1.0   
391    Úmaro Embaló  0.074425  0.081867  0.000000      1    0.0           0.0   

     A